# Data import and cleaning

### Importing packages

In [ ]:
import glob
import itertools
import math
import os
import pickle
import shutil
import warnings
from datetime import date, datetime

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from scipy import optimize, stats
from scipy.optimize import curve_fit
from scipy.signal import argrelmax, find_peaks, hilbert, peak_widths
from scipy.stats import norm
from tabulate import tabulate
from tkinter import filedialog, Tk
from uncertainties import ufloat, unumpy
from uncertainties.umath import *

from lmfit import Minimizer, Model, Parameters, report_fit
from lmfit.models import LinearModel, SplitLorentzianModel

pd.set_option('display.max_rows', 50)
warnings.filterwarnings('ignore')
plt.style.use('ggplot')
%matplotlib inline

### Selecting data folder
#### Creates result folder for output saving

In [ ]:
root = Tk()
root.withdraw()
folder_path = filedialog.askdirectory()
print('Folder path: ', folder_path)

file_path = os.path.join(folder_path, '**/*.csv')
result_folder = 'res'
folder = os.path.join(folder_path, result_folder)

if os.path.isdir(folder):
    print('Result folder already exists, cleaning...\n (If an error occurs, consider remove Result folder manually.)')
    try:
        shutil.rmtree(folder)
    except OSError as e:
        print(f"Error: {folder} : {e.strerror}")
os.mkdir(folder)
print('Result folder created.')

print('---------------------------------------------------------------------------\n')
print('Result folder path: ', folder)
print('---------------------------------------------------------------------------\n')

### Logging
log_file = os.path.join(folder, 'log.txt')

with open(log_file, 'w') as f:
    f.write(f"{datetime.now()} - {folder}")

### Acquiring individual files addresses 
#### Splits into group, sensor and time, adding a hash column used for filtering data

In [ ]:
files = pd.DataFrame(glob.glob(file_path,recursive = True),columns = ['Path'])

def extract_info(path):
    group = path.split('/')[-1].split('\\')[-3]
    sensor = path.split('/')[-1].split('\\')[-2]
    k = path.rfind('\\')
    time = int(path[k + 1:].replace('.CSV', ''))
    hash = f"{group}-{sensor}-{time}"
    return pd.Series([group, sensor, time, hash])

files[['group', 'sensor', 'time', 'hash']] = files['Path'].apply(extract_info)

addresses = files.rename(columns={'Path': 'address'}).sort_values(
    by=['group','sensor','time','address'], 
    ignore_index=True, 
    ascending=True
)

print('---------------------------------------------------------------------------\n')

text = 'Analysis'

print(text + '\n')

print('---------------------------------------------------------------------------\n')

addresses.head()

#### Provides intel on quantity of files to be scanned

In [ ]:
space = len(addresses.index)
print('Space: ', space)
today = date.today()

### Logging

files_address_list = f'{datetime.now()} - Address list - OK \n'

with open(log_file, 'a') as f:
    f.write(files_address_list)

addresses = addresses.reset_index(drop=True).set_index('address')

### Scans the addresses data frame, reads data of each file and assembles them into a combined data frame, that contains frequency and signal data
### Also, rescales frequency values to signal's order of magnitude (multiplied by $1 \times 10^{-6}$)

In [ ]:
def process_path(path):
    group = addresses.loc[path,'group']
    sensor = addresses.loc[path,'sensor']
    time = addresses.loc[path,'time']
    hashed = addresses.loc[path,'hash']

    df_import = pd.DataFrame(pd.read_csv(path, skiprows=range(0, 2)))
    df_import.drop(df_import.columns[2], axis=1, inplace=True)
    df_import.columns.values[0] = 'frequency'
    df_import.columns.values[1] = 'signal'
    df_import['group'] = group
    df_import['sensor'] = sensor                                          
    df_import['time'] = time
    df_import['hash'] = hashed

    return df_import

dfs = addresses.index.map(process_path)
combined_df = pd.concat(dfs, ignore_index=True)

combined_df['frequency'] = combined_df['frequency'] * 1.e-6
combined_df.head()

### Exports combined and addresses data frames to results folder

In [ ]:
datasets_export = f'{str(datetime.now())} - Combined and address datasets - EXPORTED \n'

with open(log_file, 'a') as f:
    f.write(datasets_export)
    f.close()
    
combined_df.to_csv(folder + '/combined_df.csv', sep=';', index=False)
addresses.to_csv(folder + '/addresses.csv', sep=';', index=False)

# Defining functions

### Split Lorentzian funcion
#### Function used to fit data to model

$$
f(x; A, \mu, \sigma, \sigma_r) = \frac{2 A}{\pi (\sigma+\sigma_r)} \big[\frac{\sigma^2}{(x - \mu)^2 + \sigma^2} * H(\mu-x) + \frac{\sigma_r^2}{(x - \mu)^2 + \sigma_r^2} * H(x-\mu)\big] + (m x + b)
$$

### Applies fitting model (Lorentzian) to each experimental dataset, retrieving a evaluation dataset based on the model. After fitting, applies an optimization function over a Monte Carlo Simulation to find the interesting point: the minimized frequency used to evaluate delocation on passing time of experiment


#### For each instance of time, plots original data, accuracy rate between Monte Carlo Simulation average frequency and minimized average frequency, and fitted curve over original data, pinpointing those values alongside minimal point from original data.

In [ ]:
def SplitLorentzianFunc(x, aux):
    # Calculate the amplitude of the Lorentzian function
    amplitude = ((2 * aux[0]) / (np.pi * (aux[2] + aux[3])))
    
    # Calculate the Lorentzian function for the lower and upper halves
    lower_half = ((aux[2] ** 2) / (((x - aux[1]) ** 2) + aux[2] ** 2))
    upper_half = ((aux[3] ** 2) / (((x - aux[1]) ** 2) + aux[3] ** 2))
    
    # Calculate the Heaviside step function for the lower and upper halves
    step_lower = np.heaviside(aux[1] - x, 0)
    step_upper = np.heaviside(x - aux[1], 0)
    
    # Calculate the linear function
    linear_func = (aux[4] * x + aux[5])

    # Return the sum of the Lorentzian function and the linear function
    return (amplitude * ((lower_half * step_lower) + (upper_half * step_upper))) + linear_func

def create_evaluate_df(x_eval, y_eval, hashed):
    evaluate_df = pd.DataFrame(x_eval, columns=['x_eval'])
    evaluate_df['y_eval'] = y_eval
    evaluate_df['hash'] = hashed
    return evaluate_df


def process_data(combined_df, log_file):
    unique_hash = combined_df.hash.unique()
    dfs_params = []
    dfs_eval = []
    frequency_shift = []

    for i in unique_hash:
        hashed = i
        xy0 = combined_df[['frequency','signal']].loc[combined_df['hash'] == hashed]
        xy0 = xy0.reset_index(drop=True)

        interval = 13
        min_value_idx = xy0.loc[xy0['signal'] == xy0['signal'].min()].index[0]
        idx = range(min_value_idx - interval, min_value_idx + interval)

        x = np.array(xy0['frequency'])[idx]
        y = np.array(xy0['signal'])[idx]
        
        # Model parametrization
        peak = SplitLorentzianModel() #prefix='slm_')
        linemod1 = LinearModel() #prefix='lm1_')
        #linemod2 = LinearModel(prefix='lm2_')
        pars = Parameters()
        pars += peak.guess(y, x=x)
        pars += linemod1.make_params(intercept=y.min(), slope=0)
        #pars += linemod2.make_params(intercept=y.min(), slope=0)
        mod = linemod1 + peak # + linemod2

        # Fit model
        result = mod.fit(y, pars, x=x)
        
        # Printing report and related information
        print(result.fit_report(min_correl=0.25))

        ### Logging

        lmfit_report = f'\n{str(datetime.now())} - LMFIT result report {i} \n' + str(result.fit_report(min_correl=0.25))

        with open(log_file, 'a') as f:
            f.write(lmfit_report)
        
        # Determining the point to be evaluated on the frequency shift   
        x_eval = np.linspace(min(x), max(x), num = 10000)
        y_eval = result.eval(result.params, x=x_eval)

        evaluate_df = pd.DataFrame(x_eval, columns=['x_eval'])
        evaluate_df['y_eval'] = y_eval
        evaluate_df['hash'] = hashed
        
        plot_x = evaluate_df['x_eval'].loc[evaluate_df['y_eval'] == evaluate_df['y_eval'].min()]
        plot_y = evaluate_df['y_eval'].loc[evaluate_df['y_eval'] == evaluate_df['y_eval'].min()]

        # Min point
        minimized_freq = []

        n = 1000
        
        for j in range(n):
            aux = np.array([
                np.random.normal( # 0
                    loc=result.params['amplitude'].value, 
                    scale=result.params['amplitude'].stderr
                ),
                np.random.normal( # 1
                    loc=result.params['center'].value, 
                    scale=result.params['center'].stderr
                ), 
                np.random.normal( # 2
                    loc=result.params['sigma'].value, 
                    scale=result.params['sigma'].stderr
                ),
                np.random.normal( # 3
                    loc=result.params['sigma_r'].value, 
                    scale=result.params['sigma_r'].stderr
                ),
                np.random.normal( # 4
                    loc=result.params['slope'].value, 
                    scale=result.params['slope'].stderr
                ),
                np.random.normal( # 5
                    loc=result.params['intercept'].value, 
                    scale=result.params['intercept'].stderr
                    )
                ])

            find_fmin = optimize.fmin(
                lambda x: SplitLorentzianFunc(x, aux),
                xy0['frequency'][min_value_idx], 
                full_output=True,
                disp=0
            )
    
            find_fmin_point = np.array([find_fmin[0].item(), find_fmin[1]])
    
            minimized_freq.append(find_fmin[0])
    
            j = j + 1

        minimized_freq = np.concatenate(minimized_freq).ravel()
        minimized_freq = minimized_freq[(minimized_freq > 0.43) & (minimized_freq < 0.444)]
        
        minimized_freq_mean = np.array(minimized_freq).mean()
        minimized_freq_std = np.array(minimized_freq).std()
        minimized_freq_std_err = minimized_freq_std / np.sqrt(n)
        freq = ufloat(minimized_freq_mean,minimized_freq_std)*1e6
        pfloat = minimized_freq_mean/plot_x.values.item()
        
        values_lst = [
            minimized_freq_mean, 
            minimized_freq_std, 
            minimized_freq_std_err,
            freq,
            hashed
        ]
        
        dfs_params.append(values_lst)
        
        print('\n')
        print('----- Results -----')
        
        def create_result_table(minimized_freq_mean, minimized_freq_std, minimized_freq_std_err, freq, pfloat):
            result_table = [
                ['Optimized frequency mean (fmin)', minimized_freq_mean],
                ['Optimized frequency standard deviation (fmin)', minimized_freq_std],
                ['Optimized frequency standard error (fmin)', minimized_freq_std_err],
                ['Optimized frequency mean with uncertainties', freq],
                ['Accuracy of estimated frequency mean / SMC',pfloat]
            ]
            return result_table
        
        result_table = create_result_table(minimized_freq_mean, minimized_freq_std, minimized_freq_std_err, freq, pfloat)
        print(str(tabulate(result_table)))
        print('\n')
        
        ### Logging
        
        variables_report = f'{datetime.now()} - Results {i} \n' + str(tabulate(result_table)) + '\n'
        
        with open(log_file, 'a') as f:
            f.write(variables_report)
            
        # PLOTING
    
        # Primary data
        plt.figure()
        plt.rcParams.update({'font.size': 18})
        ax = xy0.plot(
            x = 'frequency', 
            y = 'signal', 
            kind='scatter',
            figsize = (16,4), 
            grid=True, 
            legend=True
        )
        ax.set_title(label = 'Initial data ' + hashed, pad=20, fontdict={'fontsize':20})
        ax.set_xlabel('Frequency [MHz]')
        ax.set_ylabel('Signal')
        plt.show()
        print('\n')
        
        #Accuracy between minimized frequency mean and MCS frequency mean
        xfloat = np.linspace(0.98, 1.02, num = 100)
        yfloat = np.linspace(0, 0, num = 100)
        fig = plt.figure(figsize = (16,4))
        plt.plot(xfloat,yfloat)
        plt.plot(pfloat,0,color='k',marker='|', markersize = 15, label='Optimized frequency mean LMFIT / Optimized frequency mean SMC')
        plt.text(
            x=pfloat, 
            y=0.02, 
            s='Accuracy:    {:.8}'.format(pfloat), 
            horizontalalignment='right',
            verticalalignment='baseline'
        ) 
        plt.legend(loc='best')
        plt.title('Accuracy between frequency mean LMFIT and SMC optimized frequency mean', fontsize=20)
        plt.show()
        
        # Fit model data plot    
        fig = plt.figure(figsize = (16,8))
        plt.plot(x, y, 'o')
        plt.plot(x_eval, y_eval, 'r-', label='Best fit')
    
        plt.plot(
            xy0['frequency'][min_value_idx],
            xy0['signal'][min_value_idx],
            marker = 'D',
            color='orange', 
            markersize=8,
            label='Original data minimum frequency'
        )
        
        labels = evaluate_df['hash'].loc[evaluate_df['y_eval'] == evaluate_df['y_eval'].min()]
        
        plt.plot(
            plot_x, 
            plot_y, 
            label='Lorentz minimum frequency (LMFIT)', 
            color='green', 
            marker='s', 
            markersize=8
        )
        plt.plot(
            minimized_freq_mean, 
            plot_y, 
            label='SMC average frequency', 
            color='k', 
            marker='o', 
            markersize=8
        )
        plt.xlabel('Frequency [MHz]')
        plt.ylabel('Signal')
        plt.legend(loc='best')
        plt.title('Signal vs frequency: Lorentz function fit and points of interest '+ hashed, fontsize=20)
        plt.grid(True)
        plt.show()

        global eval_df
        
        dfs_eval.append(evaluate_df)
        
        print('Eval appended.')
        print('--------------------------------\n')
    eval_df = pd.concat(dfs_eval, ignore_index=True)
    param_df = pd.DataFrame(
        dfs_params, 
        columns=[
            'minimized_freq_mean',
            'minimized_freq_std',
            'minimized_freq_std_err',
            'freq_with_unc', 
            'hash'
        ]
    )
    return eval_df, param_df

# Running the model builder

In [ ]:
eval_df, param_df = process_data(combined_df, log_file)

### Creates a data frame with evaluated and parametrization data 

In [ ]:
def extract_info(hashed):
    group = hashed.split('-')[-3]
    sensor = hashed.split('-')[-2]
    time = hashed.split('-')[-1]
    return pd.Series([group, sensor, time])

eval_df[['group', 'sensor', 'time']] = eval_df['hash'].apply(extract_info)
eval_df['time'] = pd.to_numeric(eval_df['time'])

grouped_eval_df = eval_df.groupby(['hash']).min().sort_values(by=['group','sensor','time']).reset_index(drop=False)

complete = pd.merge(addresses, grouped_eval_df[['x_eval','y_eval','hash']], on='hash', how='left')
complete = complete.set_index(['hash']).drop('x_eval', axis=1)
complete = complete.join(param_df.set_index('hash'))
complete.to_csv(f'{folder}/complete_df.csv', sep=';')

complete_dataset = f'{datetime.now()} - Complete dataset - EXPORTED \n'

with open(log_file, 'a') as f:
    f.write(complete_dataset)

In [ ]:
complete.head(50)

### Applies a Lagergren model over minimized data in order to obtain a model for frequency decay on time.
#### Provides a detailed report on fit params of the model.
#### As result of this process, it is possilbe to estimate frequency shift of each time instance

In [ ]:
def fit_model(t, f0, a, c):
    return f0 * (1 - a * (1 - np.exp(-c * t)))

def plot_data(x, y, title, xlabel, ylabel, legend):
    plt.plot(x, y)
    plt.title(title, fontsize=20)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend(legend)
    plt.grid(True)

sensors = complete['sensor'].unique()
df_shift = pd.DataFrame(columns=['hash','shift', 'shift_value','shift_std'])
for sensor in sensors:
    g = complete['group'].loc[complete.sensor == sensor]
    if g[0] == 'C':
        label_group = ' - Control'
    else:
        label_group = ' - Test'
        
    df = complete.loc[(complete.group == g[0]) & (complete.sensor == sensor)]
    df.plot(x='time',y='minimized_freq_mean',kind='scatter',yerr='minimized_freq_std',
        figsize=(15,8),xlabel='Tempo [min]',ylabel = 'Média da frequência minimizada [MHz]',
        title = 'Sensor ' + sensor + label_group)
    plt.ticklabel_format(useOffset=False)
    plt.show()
    # Fit the function f(f0,a,c) = f0 * (1 - a * (1 - np.exp(-c * t)))
    t = df.time 
    y = df.minimized_freq_mean
    e = df.minimized_freq_std
    popt, pcov = curve_fit(fit_model, t, y, absolute_sigma=True, maxfev=100000)
    perr = np.sqrt(np.diag(pcov))
    f0 = popt[0]
    a = popt[1]
    c = popt[2]    
    gmodel = Model(fit_model)
    params = gmodel.make_params(f0=f0, a=a, c=c)
    result = gmodel.fit(y, params, t=t)    
    #Parameters with errors from LMFIT
    f0uf = ufloat(result.params['f0'].value,result.params['f0'].stderr)
    auf = ufloat(result.params['a'].value,result.params['a'].stderr)
    cuf = ufloat(result.params['c'].value,result.params['c'].stderr)
    shifts = []
    shifts_values = []
    shifts_std = []
    for k in range(len(df)):
        tk  = ufloat(df.time[k],df.minimized_freq_std[k])
        ft = f0uf * (1 - auf * (1 - exp(-cuf * tk))) 
        shift = ft*1e6 - f0uf*1e6
        shift_value = shift.nominal_value
        shift_std = shift.std_dev
        shifts.append(shift)
        shifts_values.append(shift_value)
        shifts_std.append(shift_std)
    
    df_aux           = pd.DataFrame(columns=['hash','shift', 'shift_value','shift_std'])
    df_aux['hash']   = df.index
    df_aux['shift']  = shifts
    df_aux['shift_value']  = shifts_values
    df_aux['shift_std']  = shifts_std
    df_shift = pd.concat([df_shift, df_aux],ignore_index=True)
    t30  = ufloat(df.time[-1],df.minimized_freq_std[-1])
    f_t30 = f0uf * (1 - auf * (1 - exp(-cuf * t30)))
    shift30 = f_t30*1e6 - f0uf*1e6
    # Plot
    plt.figure(figsize = (18,8))
    ax = plt.axes()
    ax.scatter(t, y, label='Raw data')
    ax.errorbar(t, y, yerr=e,fmt="o")
    ax.plot(t,fit_model(t, *popt),'k',label=f'Fitted curve: f0={f0:.4f}, a={a:.4f}, c={c:.4f}')
    ax.set_title(f'Lagergren - Sensor {sensor} {label_group}')
    ax.set_ylabel('Minimized frequency mean [MHz]')
    ax.set_xlabel('Time')
    ax.legend()
    ax.ticklabel_format(useOffset=False)
    plt.legend()
    plt.text(x=min(t),y=min(y),s=f'Frequence shift [Hz]:    {shift30:.8u}',horizontalalignment='left',verticalalignment='baseline') 
    plt.show()   
    print(f'Summary - Sensor {sensor} {label_group}')
    result_table2 = [
        ['Parameter f0', f'{f0uf:.4u}'],
        ['Parameter a', f'{auf:.4u}'],
        ['Parameter c', f'{cuf:.4u}'],
        ['Frequency t = 30 [MHz]', f'{f_t30:.4u}'],
        ['Frequency shift [Hz]',f'{shift30:.8u}']
    ]
    print(str(tabulate(result_table2)))  
    df_values = df[['group','sensor','time','y_eval','minimized_freq_mean','minimized_freq_std']]
    headers = ['Group','Sensor','Time','Signal fit','Minimized frequency mean [MHz]','Std Dev']
    tablefmt='psql'
    values_table = tabulate(df_values,headers=headers,tablefmt=tablefmt)
    print(values_table)
    print('\n')
    print(result.fit_report())

df_shift = df_shift.set_index(['hash'])
complete_shifts = pd.merge(complete[['group','sensor','time']],df_shift,on='hash', how='left')

In [ ]:
### Logging
complete_shifts_export = f'\n {str(datetime.now())} - Complete shifts list - EXPORTED \n'

with open(log_file, 'a') as f:
    f.write(complete_shifts_export)
    f.close()
    
complete_shifts.head()

In [ ]:
def plot_shift(data, hue, title):
    plt.figure(figsize = (18,8))
    sns.lineplot(
        data=data, 
        x='time', 
        y='shift_value', 
        hue=hue
    )
    plt.title(title)
    plt.ylabel('Nominal shift f(t) - f(0) [MHz]')
    plt.xlabel('Time [min]')
    plt.show()

plot_shift(complete_shifts, 'sensor', 'Frequency shift by sensor')
plot_shift(complete_shifts, 'group', 'Frequency shift by group')

In [ ]:
plt.figure(figsize = (18,10))
sns.boxplot(x='group', y='shift_value', data=complete_shifts, hue= 'group')
sns.swarmplot(x='group', y='shift_value', data=complete_shifts, size=7, hue = 'time')
plt.title('Frequency shift by group')
plt.ylabel('Nominal shift f(t) - f(0) [MHz]')
plt.xlabel('Group')
plt.show()

### Splits data to be used bootstrapped. This dataset is used on the classifier as well

In [ ]:
complete_shifts_clf = complete_shifts.drop(['sensor','shift'],axis=1).reset_index(drop=True)
complete_shifts_clf = complete_shifts_clf[complete_shifts_clf.time != 0]
complete_shifts_clf = complete_shifts_clf.reset_index(drop=True)

In [ ]:
complete_shifts_clf.head(25)

In [ ]:
c_list_to_bootstrap = complete_shifts_clf[complete_shifts_clf['group'] == 'C'].values.tolist()
c_list_to_bootstrap

In [ ]:
t_list_to_bootstrap = complete_shifts_clf[complete_shifts_clf['group'] == 'T'].values.tolist()
t_list_to_bootstrap

#### Bootstrapping by group

In [ ]:
import random

n = 1000

# Initialize a list to store the control data
c_data = []
t_data = []

# Generate n random data points based on the minimized control data
for _ in range(n):
    c_original = random.choice(c_list_to_bootstrap)
    c_data.append(c_original)
    
    t_original = random.choice(t_list_to_bootstrap)
    t_data.append(t_original)


# Shuffle the data
random.shuffle(c_data)
random.shuffle(t_data)

complete_list = c_data + t_data
random.shuffle(complete_list)

# Example of the first 10 data points
for i in range(5):
    print(c_data[i])
    print(t_data[i])
    print(complete_list[i])
    print('----------------------\n')

### Classification model
#### A set of different classifiers is applied to verify metrics and select best models to be used 

In [ ]:
bootstrapped = pd.DataFrame(complete_list, columns = ['group', 'time', 'frequency','standard deviation']) 

### Logging
bootstrapped_shifts_export = f'\n {str(datetime.now())} - Bootstrapped list - OK \n'

with open(log_file, 'a') as f:
    f.write(bootstrapped_shifts_export)
    f.close()

bootstrapped

In [ ]:
fig, axs = plt.subplots(ncols=2, figsize= (18,10))
sns.violinplot(x='group', y='frequency', data=bootstrapped, hue= 'group', inner="quart", ax=axs[0])
sns.boxplot(x='group', y='frequency', data=bootstrapped, hue='time', notch=False, ax=axs[1])
axs[0].set_xlabel(None)
axs[1].set_xlabel(None)
axs[0].set_ylabel(None)
axs[1].set_ylabel(None)
fig.suptitle('Frequency shift by group and time - Bootstrapped')
fig.supxlabel('Group')
fig.supylabel('Nominal shift f(t) - f(0) [MHz]')
plt.show()

plt.figure(figsize = (18,10))
sns.violinplot(x='group', y='frequency', data=bootstrapped, hue= 'group', inner="quart")
sns.boxplot(x='group', y='frequency', data=bootstrapped, hue='time', notch=True)
plt.title('Frequency shift by group by time - Bootstrapped')
plt.ylabel('Nominal shift f(t) - f(0) [MHz]')
plt.xlabel('Group')
plt.show()

In [ ]:
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

def plot_confusion_matrix(conf_matrix, name):
    plt.figure(figsize=(5, 2))
    sns.heatmap(
        conf_matrix, 
        annot=True, 
        fmt='d', 
        cmap='Blues', 
        xticklabels=["C", "T"], 
        yticklabels=["C", "T"]
    )
    plt.title(f"Confusion matrix - {name}", fontsize=10)
    plt.xlabel("Predicted",fontsize=10)
    plt.ylabel("True",fontsize=10)
    plt.show()

# Define values to classifier
X = np.array([(time, shift_value, shift_std) for _, time, shift_value, shift_std in complete_list])
y = np.array([group for group, _, _, _ in complete_list])

# Train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=0)

# Standardize data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

classifiers = {
    "Logistics Regression": LogisticRegression(max_iter=1000),
    "Decision Tree": DecisionTreeClassifier(),
    "Random Forest": RandomForestClassifier(),
    "AdaBoost": AdaBoostClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "Naive Bayes": GaussianNB(),
    "K-Nearest Neighbors": KNeighborsClassifier(),
    "SVM (SVC)": SVC(kernel='linear'),
    "SVM (RBF)": SVC(kernel='rbf'),
    "SVM (Poly)": SVC(kernel='poly')
}

# Metrics
metrics_table = []

# Run classifier
for name, clf in classifiers.items():
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    
    # metrics calculate
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred, output_dict=True)
    
    # cross val
    scores = cross_val_score(clf, X, y, cv=5, scoring='accuracy')
    
    # Add the metrics to the table
    metrics_table.append({
        'Algorithm': name,
        'CV Avg Accuracy': scores.mean(),
        'CV Acc std dev': scores.std(),
        'Accuracy': accuracy,
        'Precision': report['weighted avg']['precision'],
        'Recall': report['weighted avg']['recall'],
        'F1-Score': report['weighted avg']['f1-score'],
        'Support': report['weighted avg']['support'],
    })

    conf_matrix = confusion_matrix(y_test, y_pred)
    plot_confusion_matrix(conf_matrix, name)
    
    # print cross val metrics
    print(f"Classifier: {name}")
    print(f"Accuracy avg: {scores.mean()}")
    print(f"Accuracy 5 runs: {scores}")
    print("\n")
    print('-----------------------------------------------------------------------------------------------------\n')

    metrics_table_export = [
        ['Classifier', name],
        ['Accuracy avg', scores.mean()]
    ]
    ### Logging
    metrics_list_export = f'\n {str(datetime.now())} - Metrics list - EXPORTED \n'
    with open(log_file, 'a') as f:
        f.write(metrics_list_export)
        f.write(str(tabulate(metrics_table_export)))

# Convert the table into a Pandas DataFrame
metrics_df = pd.DataFrame(metrics_table)

In [ ]:
metrics_df = metrics_df.sort_values(by='CV Avg Accuracy', ascending=False)

metrics_df

In [ ]:
### Logging
all_metrics_df = tabulate(metrics_df, headers='keys', tablefmt='psql')

all_metrics = f'{str(datetime.now())} - All metrics \n' + str(all_metrics_df) + '\n'

with open(log_file, 'a') as f:
    f.write(all_metrics)

#### Model export

In [ ]:
import joblib

# Save the model
model_filename = f"{folder}/model_classifier.pkl"
joblib.dump(clf, model_filename)

# Log the export
model_file_export = f"\n{datetime.now()} - Model file exported. Analysis is complete!"

with open(log_file, 'a') as f:
    f.write(model_file_export)